<a href="https://colab.research.google.com/github/tyoc213/fastai_xla_extensions/blob/explorations1/explore_nbs/ColaboratoryFilteringTPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
VERSION = "20200707" #"nightly"  #"20200515" @param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py > /dev/null
!python pytorch-xla-env-setup.py --version $VERSION > /dev/null
#import torch_xla.core.xla_model as xm

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  73071      0 --:--:-- --:--:-- --:--:-- 73071
Copying gs://tpu-pytorch/wheels/torch-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
- [1 files][107.5 MiB/107.5 MiB]                                                
Operation completed over 1 objects/107.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
\ [1 files][123.8 MiB/123.8 MiB]                                                
Operation completed over 1 objects/123.8 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-nightly+20200707-cp36-cp36m-linux_x86_64.whl...
/ [1 files][  2.2 MiB/  2.2 MiB]                                                
Operation completed over 1 objects/2.2 MiB.                                      
ERROR:

In [2]:
!pip install https://github.com/butchland/fastai_xla_extensions/archive/master.zip > /dev/null
import fastai_xla_extensions.core

In [3]:
#!pip install fastai2 > /dev/null

In [4]:
from fastai2.tabular.all import *
from fastai2.collab import *
dede = default_device()
dede

device(type='xla', index=1)

In [5]:
path = untar_data(URLs.ML_100k)
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      usecols=(0,1,2), names=['user','movie','rating'])
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)

In [6]:
ratings = ratings.merge(movies)

In [7]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64, device=dede)
dls.device

device(type='xla', index=1)

In [8]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))

In [9]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.961844,0.957353,00:41
1,0.859439,0.872807,00:38
2,0.742559,0.834572,00:38
3,0.592265,0.821978,00:38
4,0.474402,0.823461,00:39


/usr/local/lib/python3.6/dist-packages/fastai2/callback/schedule.py:68: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  idx = (pos >= pcts).nonzero().max()


### Interpretation

In [14]:
g = ratings.groupby("title")['rating'].count()
top_movies = g.sort_values(ascending=False).index.values[:1000]
top_movies[:10]

array(['Star Wars (1977)', 'Contact (1997)', 'Fargo (1996)',
       'Return of the Jedi (1983)', 'Liar Liar (1997)',
       'English Patient, The (1996)', 'Scream (1996)', 'Toy Story (1995)',
       'Air Force One (1997)', 'Independence Day (ID4) (1996)'],
      dtype=object)

In [15]:
movie_bias = learn.model.bias(top_movies, is_item=True)
movie_bias.shape

torch.Size([1000])

In [16]:
mean_ratings = ratings.groupby("title")['rating'].mean()
movie_ratings = [(b, i, mean_ratings.loc[i]) for i,b in zip(top_movies,movie_bias)]

In [17]:
item0 = lambda o:o[0]
sorted(movie_ratings, key=item0)[:15]

[(tensor(-0.3617),
  'Children of the Corn: The Gathering (1996)',
  1.3157894736842106),
 (tensor(-0.3318),
  'Lawnmower Man 2: Beyond Cyberspace (1996)',
  1.7142857142857142),
 (tensor(-0.2724), 'Mortal Kombat: Annihilation (1997)', 1.9534883720930232),
 (tensor(-0.2705), 'Crow: City of Angels, The (1996)', 1.9487179487179487),
 (tensor(-0.2705), "McHale's Navy (1997)", 2.1884057971014492),
 (tensor(-0.2593), 'Home Alone 3 (1997)', 1.894736842105263),
 (tensor(-0.2576), 'Kansas City (1996)', 2.260869565217391),
 (tensor(-0.2525), 'Free Willy 3: The Rescue (1997)', 1.7407407407407407),
 (tensor(-0.2453), 'Cable Guy, The (1996)', 2.339622641509434),
 (tensor(-0.2428), 'Leave It to Beaver (1997)', 1.8409090909090908),
 (tensor(-0.2403), 'Bio-Dome (1996)', 1.903225806451613),
 (tensor(-0.2297),
  'Candyman: Farewell to the Flesh (1995)',
  2.4285714285714284),
 (tensor(-0.2053), 'Hellraiser: Bloodline (1996)', 2.0),
 (tensor(-0.1985), "Joe's Apartment (1996)", 2.2444444444444445),
 (ten